In [70]:
!pip install deap pandas numpy

In [71]:
import random
import numpy as np
import pandas as pd
from deap import base, creator, tools, algorithms
import json

In [72]:
flows = pd.read_csv("flows_peak.csv")
with open("constraints.json") as f:
    constraints = json.load(f)

In [73]:
# === Настройки ===
NUM_INTERSECTIONS = 3
TRAVEL_TIME = 30  # секунд между перекрёстками
CYCLE_FIXED = 120
GENE_SIZE = 3  # green_main, green_sec, offset
CHROMOSOME_SIZE = NUM_INTERSECTIONS * GENE_SIZE

In [74]:
# === Целевая функция с весом задержки от bus_share ===
def fitness(individual):
    total_delay = 0

    for i in range(NUM_INTERSECTIONS):
        idx = i * GENE_SIZE
        green_main = individual[idx]
        green_sec = individual[idx + 1]
        offset = individual[idx + 2]

        inter_id = i + 1
        int_flows = flows[flows['intersection_id'] == inter_id]

        for _, row in int_flows.iterrows():
            is_main = row['approach'] in ['N', 'S']
            flow = row['intensity_veh_per_hr']
            bus_share = row['bus_share']
            green = green_main if is_main else green_sec

            # Безусловный приоритет автобусам (по всем направлениям)
            bus_bonus = constraints["min_extra_green_sec"]
            effective_green = min(CYCLE_FIXED, green + bus_bonus)

            # Весовая задержка: больший вес — если много автобусов
            weight = 1 + 2 * bus_share
            delay = (CYCLE_FIXED - effective_green) * flow * weight

            total_delay += delay

    # Пенальти за рассинхрон offset (зелёная волна)
    offset_penalty = 0
    for i in range(1, NUM_INTERSECTIONS):
        prev = individual[(i - 1) * GENE_SIZE + 2]
        curr = individual[i * GENE_SIZE + 2]
        expected = prev + TRAVEL_TIME
        offset_penalty += abs(curr - expected)

    return total_delay + 1000 * offset_penalty,


In [75]:
# === Проверка ограничений ===
def check_constraints(ind):
    for i in range(NUM_INTERSECTIONS):
        idx = i * GENE_SIZE
        g1, g2, o = ind[idx:idx+3]
        lost = 2 * constraints["lost_time_sec_per_phase"]

        if g1 < constraints["pedestrian_green_sec"] or g2 < constraints["pedestrian_green_sec"]:
            return False
        if g1 + g2 + lost > CYCLE_FIXED:
            return False
        if not (0 <= o <= CYCLE_FIXED):
            return False
    return True


In [76]:
# === Инициализация GA ===
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

c:\Users\borze\anaconda3\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
c:\Users\borze\anaconda3\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [77]:
def random_signal_fixed_cycle():
    g1 = random.randint(constraints['min_green_sec'], 60)
    g2 = random.randint(constraints['min_green_sec'], 60)
    lost = 2 * constraints["lost_time_sec_per_phase"]
    while g1 + g2 + lost > CYCLE_FIXED:
        g1 -= 1
    offset = random.randint(0, CYCLE_FIXED)
    return [g1, g2, offset]
toolbox.register("individual", tools.initIterate, creator.Individual, 
                 lambda: sum([random_signal_fixed_cycle() for _ in range(NUM_INTERSECTIONS)], []))
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", fitness)
toolbox.register("mate", tools.cxBlend, alpha=0.4)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=5, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

In [78]:
def feasible(ind):
    return check_constraints(ind)

toolbox.decorate("evaluate", tools.DeltaPenalty(feasible, 1e6))

In [79]:
# === Запуск GA ===
def run_ga():
    pop = toolbox.population(n=100)
    hof = tools.HallOfFame(1)

    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("min", np.min)

    algorithms.eaSimple(pop, toolbox, cxpb=0.7, mutpb=0.2, ngen=50, 
                        stats=stats, halloffame=hof, verbose=True)

    best = hof[0]
    print("\n=== ЛУЧШЕЕ РЕШЕНИЕ (ЦИКЛ 120 СЕК) ===")
    for i in range(NUM_INTERSECTIONS):
        idx = i * GENE_SIZE
        g1, g2, o = best[idx:idx+3]
        print(f"Перекрёсток {i+1}: цикл=120, главная={g1:.0f}, второстепенная={g2:.0f}, offset={o:.0f}")
    print("Оценка:", fitness(best)[0])

In [80]:
if __name__ == "__main__":
    run_ga()

gen	nevals	avg   	min   
0  	100   	845729	676885
1  	75    	832886	677960
2  	68    	813700	658984
3  	73    	791067	609559
4  	82    	756896	609559
5  	79    	754162	604815
6  	70    	758750	604815
7  	71    	711368	581391
8  	75    	719190	582901
9  	76    	684724	582901
10 	81    	663757	567954
11 	75    	686894	567954
12 	85    	690463	563575
13 	55    	665296	555868
14 	70    	638372	548273
15 	75    	663635	539126
16 	73    	660861	539126
17 	86    	641145	539126
18 	82    	619100	539126
19 	73    	635877	540963
20 	72    	603352	536174
21 	71    	595732	534864
22 	64    	602437	533455
23 	59    	592052	533827
24 	71    	608549	529787
25 	74    	598199	529127
26 	60    	578212	523025
27 	74    	604570	525365
28 	78    	645598	525282
29 	74    	631195	524358
30 	74    	601167	524358
31 	69    	585879	523165
32 	75    	584570	523165
33 	75    	588815	523134
34 	84    	607090	521645
35 	69    	615287	520559
36 	89    	624896	520725
37 	69    	613866	520725
38 	75    	599498	520048


In [81]:
# === Входные данные ===
flows = pd.read_csv("flows_peak.csv")
signals = pd.read_csv("signals_current.csv")

with open("constraints.json") as f:
    constraints = json.load(f)

total_delay = 0


In [82]:
for _, row in flows.iterrows():
    inter_id = row['intersection_id']
    approach = row['approach']
    flow = row['intensity_veh_per_hr']
    bus_share = row['bus_share']

    sig = signals[signals['intersection_id'] == inter_id].iloc[0]

    cycle = sig['cycle_sec']
    green_main = sig['green_main_sec']
    green_sec = sig['green_secondary_sec']

    # Направление
    is_main = approach in ['N', 'S']
    green = green_main if is_main else green_sec

    # Приоритет автобусов
    bus_bonus = constraints["min_extra_green_sec"]
    effective_green = min(cycle, green + bus_bonus)

    delay = (cycle - effective_green) * flow
    total_delay += delay

print(f"💡 Общая задержка ДО оптимизации: {total_delay:.2f}")


💡 Общая задержка ДО оптимизации: 690500.00
